# Quantium Virtual Internship

## The task
We need to analyse the data to understand current purchasing trends and behaviours. The client is particularly interested in customer segments and their chip purchasing behaviour. Consider what metrics would help describe the customers’ purchasing behaviour.  

# EDA
To get started, download the resource csv data files below and begin performing high level data checks such as:
Creating and interpreting high level summaries of the data
Finding outliers and removing these (if applicable)
Checking data formats and correcting (if applicable)

You will also want to derive extra features such as pack size and brand name from the data.  Remember our end goal is to form a strategy based on the findings to provide a clear recommendation to Julia the Category Manager so make sure your insights can have a commercial application.

Examine transaction data – look for inconsistencies, missing data across the data set, outliers, correctly identified category items, numeric data across all tables. 

Examine customer data – check for similar issues in the customer data, look for nulls and when you are happy merge the transaction and customer data together so it’s ready for the analysis ensuring you save your files along the way.

Data analysis and customer segments – in your analysis make sure you define the metrics – look at total sales, drivers of sales, where the highest sales are coming from etc. Explore the data, create charts and graphs as well as noting any interesting trends and/or insights you find. These will all form part of our report to Julia. 

Deep dive into customer segments – define your recommendation from your insights, determine which segments we should be targeting, if packet sizes are relative and form an overall conclusion based on your analysis. 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [31]:
df = pd.read_excel('data/QVI_transaction_data.xlsx')
df.head()

,DATE,STORE_NBR,LYLTY_CARD_NBR,TXN_ID,PROD_NBR,PROD_NAME,PROD_QTY,TOT_SALES
0,43390,1,1000,1,5,Natural Chip Compny SeaSalt175g,2,6.0
1,43599,1,1307,348,66,CCs Nacho Cheese 175g,3,6.3
2,43605,1,1343,383,61,Smiths Crinkle Cut Chips Chicken 170g,2,2.9
3,43329,2,2373,974,69,Smiths Chip Thinly S/Cream&Onion 175g,5,15.0
4,43330,2,2426,1038,108,Kettle Tortilla ChpsHny&Jlpno Chili 150g,3,13.8


In [32]:
df.shape

(264836, 8)

In [33]:
# function to get the weight of pack
def get_weight(name):
    if name.startswith('Kettle 135g'):
        return name[7:10].strip()
    else:
        return name[-4:-1].strip()

In [34]:
df['weight'] = df['PROD_NAME'].apply(lambda x: get_weight(x))

In [35]:
# change weight data type to integer
df['weight'] = df['weight'].astype(int)

In [36]:
# get brand name
df['brand'] = df['PROD_NAME'].apply(lambda x: x.split()[0])

In [103]:
# check brand names
df['brand'].value_counts()

Kettle             41288
Smiths             28860
Pringles           25102
Doritos            22041
Infuzions          14201
Thins              14075
RRD                11894
WW                 10320
Cobs                9693
Tostitos            9471
Twisties            9454
Natural Chip Co     7469
Tyrrells            6442
Grain               6272
Red Rock Deli       5885
Cheezels            4603
CCs                 4551
Smith               4527
Sunbites            4476
Dorito              3185
Cheetos             2927
Woolworths          1516
French Fries        1418
Name: brand, dtype: int64

In [101]:
df[df['PROD_NAME'].str.contains('French')]['PROD_NAME'].value_counts()

Smiths Crinkle Cut  French OnionDip 150g    1438
French Fries Potato Chips 175g              1418
Name: PROD_NAME, dtype: int64

In [69]:
df[(df['brand']=='Woolworths') & (df['PROD_NAME'].str.contains('Salsa'))]['PROD_NAME'].value_counts()

Series([], Name: PROD_NAME, dtype: int64)

In [40]:
# rename brand name
df.loc[df['PROD_NAME'].str.startswith('Red'), ['brand']] = 'Red Rock Deli'

In [81]:
# rename brand name
df.loc[df['PROD_NAME'].str.startswith('Natural'), ['brand']] = 'Natural Chip Co'

In [88]:
# rename brand name
df.loc[df['PROD_NAME'].str.startswith('Infzns'), ['brand']] = 'Infuzions'

In [92]:
# rename brand name
df.loc[df['PROD_NAME'].str.startswith('Snbts'), ['brand']] = 'Sunbites'

In [94]:
# rename brand name
df.loc[df['PROD_NAME'].str.startswith('Burger'), ['brand']] = 'Smith'

In [97]:
# rename brand name
df.loc[df['PROD_NAME'].str.startswith('GrnWves'), ['brand']] = 'Sunbites'

In [100]:
# rename brand name
df.loc[df['PROD_NAME'].str.startswith('NCC'), ['brand']] = 'Natural Chip Co'

In [102]:
# rename brand name
df.loc[df['PROD_NAME'].str.startswith('French'), ['brand']] = 'French Fries'

#### Wrong categories

In [41]:
# dip is just a name for chip flavor, not wrongly categorised
df[(df['PROD_NAME'].str.contains('Dip')) & (df['brand']=='Smiths')]['PROD_NAME'].value_counts()

Smiths Crinkle Cut  French OnionDip 150g    1438
Name: PROD_NAME, dtype: int64

In [42]:
# Old El Paso Salsa Dips are wrongly categories under chips
df[df['PROD_NAME'].str.startswith('Old')]['PROD_NAME'].value_counts()

Old El Paso Salsa   Dip Chnky Tom Ht300g    3125
Old El Paso Salsa   Dip Tomato Med 300g     3114
Old El Paso Salsa   Dip Tomato Mild 300g    3085
Name: PROD_NAME, dtype: int64

In [44]:
# identify all Old El Paso dip products
old_el = df[df['PROD_NAME'].str.startswith('Old')].index

In [45]:
# remove Old El Paso products
df = df.drop(old_el)

In [54]:
# Doritos salsas wrongly categorised
df[(df['brand']=='Doritos') & (df['PROD_NAME'].str.contains('Salsa'))]['PROD_NAME'].value_counts()

Doritos Salsa Mild  300g           1472
Doritos Salsa       Medium 300g    1449
Name: PROD_NAME, dtype: int64

In [58]:
# identify Doritos salsas
doritos_salsa = df[(df['brand']=='Doritos') & (df['PROD_NAME'].str.contains('Salsa'))].index

In [59]:
# remove Doritos salsas
df = df.drop(doritos_salsa)

In [64]:
# Woolworth salsas wrongly categorised
df[(df['brand']=='Woolworths') & (df['PROD_NAME'].str.contains('Salsa'))]['PROD_NAME'].value_counts()

Woolworths Mild     Salsa 300g    1491
Woolworths Medium   Salsa 300g    1430
Name: PROD_NAME, dtype: int64

In [66]:
# identify Woolworth salsas
woolworth_salsa = df[(df['brand']=='Woolworths') & (df['PROD_NAME'].str.contains('Salsa'))]['PROD_NAME'].index

In [67]:
# remove Doritos salsas
df = df.drop(woolworth_salsa)

## Customer data

In [4]:
df2 = pd.read_csv('data/QVI_purchase_behaviour.csv')
df2.head()

,LYLTY_CARD_NBR,LIFESTAGE,PREMIUM_CUSTOMER
0,1000,YOUNG SINGLES/COUPLES,Premium
1,1002,YOUNG SINGLES/COUPLES,Mainstream
2,1003,YOUNG FAMILIES,Budget
3,1004,OLDER SINGLES/COUPLES,Mainstream
4,1005,MIDAGE SINGLES/COUPLES,Mainstream


In [5]:
df2.shape

(72637, 3)

In [8]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72637 entries, 0 to 72636
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   LYLTY_CARD_NBR    72637 non-null  int64 
 1   LIFESTAGE         72637 non-null  object
 2   PREMIUM_CUSTOMER  72637 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.7+ MB
